In [244]:
from random import randint

In [245]:
def fast_exp(a: int, b: int) -> int:
  if a < 0 or b < 0:
    raise Exception('Invalid parameters values')
  if a == 0 and b == 0:
    raise Exception('Invalid parameters value (0 to 0 exponentiation)')
  
  if b == 0:
    return 1
  elif b % 2 == 0:
    t = fast_exp(a, b // 2)
    return t*t
  else:
    t = fast_exp(a, (b-1) // 2)
    return t*t*a

In [246]:
def exp_mod(a: int, b: int, n: int) -> int:
  if a < 0 or b < 0 or n <= 0:
    raise Exception('Invalid parameters values')
  if a == 0 and b == 0:
    raise Exception('Invalid parameters value (0 to 0 exponentiation)')

  if b == 0:
    return 1
  elif b % 2 == 0:
    t = exp_mod(a, b // 2, n)
    return (t*t) % n
  else:
    t = exp_mod(a, (b-1) // 2, n)
    return (t*t*a) % n


In [247]:
## Falta ver la cantidad de b_k que se generan, para que estén no repetidas

def miller_rabin(n: int, k: int) -> bool:
  if n < 1 or k < 1:
    raise Exception('Invalid parameters values')
  if n == 2 or n == 3:
    return True
  if k >= n - 2:
    raise Exception(
        'k value is to high for given n (k must be lesser than n - 2)'
    )

  m = n - 1
  l = 0
  while m % 2 == 0:
    l += 1
    m = m//2

  for i in range(k):
    b = randint(2, n - 2)
    x = exp_mod(b, m, n)
    if x == 1 or x == n - 1:
      continue
    continue_bool = False
    for j in range(l - 1):
      x = exp_mod(x, 2, n)
      if x == n - 1:
        continue_bool = True
        break
    if continue_bool:
      continue
    return False
  return True

def generar_primo(l: int) -> int:
  if l < 1:
    raise Exception('Invalid parameter value')
  min_value = fast_exp(10, l-1)
  max_value = fast_exp(10, l)-1

  random_number = randint(min_value, max_value)

  while not miller_rabin(random_number, 100):
    random_number = randint(min_value, max_value)
  return random_number

In [248]:
def alg_ext_euclides(a: int, b: int) -> (int, int, int):
  if a < b or b < 0 or a <= 0:
    raise Exception('Invalid parameters values')
  def recursion(n1: int, n2: int) -> (int, int, int):
    if n1 == 0:
      return (n2, 0, 1)

    mcd = recursion(n2 % n1, n1)

    s0 = mcd[1]
    t0 = mcd[2]

    s = t0 - n2//n1 * s0
    t = s0

    return (mcd[0], s, t)
  return recursion(a, b)

In [249]:
def inverso(a: int, n: int) -> int:
  if a < 1 or n < 2:
    raise Exception('Invalid parameters values')
  
  if a < n:
    mcd = alg_ext_euclides(n, a)
    if mcd[0] != 1:
      raise Exception('The numbers are not relative primes')
    if mcd[2] >= 0:
      return mcd[2]
    else:
      return mcd[2] + n
  else:
    mcd = alg_ext_euclides(a, n)
    if mcd[0] != 1:
      raise Exception('The numbers are not relative primes')
    if mcd[1] >= 0:
      return mcd[1]
    else:
      return mcd[1] + n

In [256]:
def generar_clave(l: int):
  if l < 0:
    raise Exception("Length of the keys must be positive")
  P = generar_primo(l)
  Q = generar_primo(l)
  while P == Q:
    Q = generar_primo(l)
  N = P*Q
  Phi_N = (P - 1)*(Q - 1)

  d = randint(0, Phi_N - 1)

  mcd = alg_ext_euclides(Phi_N, d)
  while mcd[0] != 1:
    d = randint(0, Phi_N - 1)
    mcd = alg_ext_euclides(Phi_N, d)

  e = inverso(d, Phi_N)

  with open("private_key.txt", "w") as private_file:
    private_file.write(str(d))
    private_file.write("\n")
    private_file.write(str(N))

  with open("public_key.txt", "w") as public_file:
    public_file.write(str(e))
    public_file.write("\n")
    public_file.write(str(N))

In [268]:
def enc(m: int) -> int:
  with open("public_key.txt", "r") as public_file:
    lines = public_file.readlines()
    e = int(lines[0])
    N = int(lines[1])
  return exp_mod(m, e, N)

def dec(m: int) -> int:
  with open("private_key.txt", "r") as private_file:
    lines = private_file.readlines()
    d = int(lines[0])
    N = int(lines[1])
  return exp_mod(m, d, N)

In [267]:
enc_value = enc(52923859225)

dec_value = dec(enc_value)

print(f"Enc number: {enc_value}")
print(f"Dec number: {dec_value} (the original!!)")

Enc number: 17331194810845433452251172182143093143542947333437078333036652211632781452263763049219842923783273524636682121317223263576285691677583500362067957203729385731655967101672479385720534577373823857255581
Dec number: 52923859225 (the original!!)
